<a href="https://colab.research.google.com/github/KBVKarthik/Google-Colaboratory-Notebooks/blob/main/Zero_Shot_Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install contextualized-topic-models
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -U tqdm

In [2]:
!wget https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt

--2022-03-09 10:28:32--  https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6208417 (5.9M) [text/plain]
Saving to: ‘dbpedia_sample_abstract_20k_unprep.txt.1’

dbpedia_sample_abst 100%[===================>]   5.92M  --.-KB/s    in 0.08s   

2022-03-09 10:28:32 (77.1 MB/s) - ‘dbpedia_sample_abstract_20k_unprep.txt.1’ saved [6208417/6208417]



In [3]:
!head -n 1 dbpedia_sample_abstract_20k_unprep.txt

The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry


In [4]:
text_file = "dbpedia_sample_abstract_20k_unprep.txt"

In [5]:
from contextualized_topic_models.models.ctm import ZeroShotTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
import nltk

In [6]:
nltk.download('stopwords')

documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()]
sp = WhiteSpacePreprocessing(documents, stopwords_language='english')
preprocessed_documents, unpreprocessed_corpus, vocab = sp.preprocess()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [7]:
preprocessed_documents[:2]

['mid peninsula highway proposed across peninsula canadian province ontario although highway connecting hamilton fort south international study published ministry',
 'died march american photographer specialized photography operated studio silver spring maryland later lived florida magazine photographer year']

In [8]:
tp = TopicModelDataPreparation("distiluse-base-multilingual-cased")

training_dataset = tp.create_training_set(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

100%|██████████| 504M/504M [00:38<00:00, 13.0MB/s]


Batches:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [9]:
tp.vocab[:10]

['abbreviated',
 'academic',
 'academy',
 'access',
 'according',
 'achieved',
 'acquired',
 'acre',
 'acres',
 'across']

In [10]:
ctm = ZeroShotTM(input_size=len(tp.vocab), bert_input_size=512, n_components=50, num_epochs=15)
ctm.fit(training_dataset)

Epoch: [15/15]	 Seen Samples: [300000/300000]	Train Loss: 138.7840030029297	Time: 0:00:10.130738: : 15it [02:33, 10.22s/it]


In [11]:
ctm.get_topic_lists(5)

[['born', 'american', 'former', 'new', 'university'],
 ['family', 'found', 'native', 'species', 'genus'],
 ['member', 'politician', 'party', 'born', 'november'],
 ['company', 'software', 'based', 'system', 'products'],
 ['family', 'found', 'species', 'native', 'mm'],
 ['world', 'born', 'olympics', 'summer', 'competed'],
 ['king', 'century', 'son', 'roman', 'greek'],
 ['played', 'league', 'football', 'season', 'team'],
 ['river', 'island', 'bridge', 'peak', 'line'],
 ['film', 'directed', 'series', 'written', 'produced'],
 ['world', 'born', 'held', 'event', 'tournament'],
 ['played', 'league', 'football', 'born', 'professional'],
 ['born', 'known', 'best', 'english', 'composer'],
 ['term', 'used', 'theory', 'usually', 'type'],
 ['company', 'founded', 'organization', 'based', 'service'],
 ['north', 'west', 'south', 'km', 'mi'],
 ['house', 'built', 'story', 'historic', 'located'],
 ['system', 'used', 'usually', 'systems', 'use'],
 ['album', 'released', 'band', 'rock', 'studio'],
 ['league'

In [12]:
topics_predictions = ctm.get_thetas(training_dataset, n_samples=5)

/usr/local/lib/python3.7/dist-packages/contextualized_topic_models/models/ctm.py:257: DeprecationWarning: Call to `get_thetas` is deprecated and will be removed in version 2, use `get_doc_topic_distribution` instead
  DeprecationWarning)
Sampling: [5/5]: : 5it [00:41,  8.32s/it]


In [13]:
preprocessed_documents[0]

'mid peninsula highway proposed across peninsula canadian province ontario although highway connecting hamilton fort south international study published ministry'

In [14]:
import numpy as np
topic_number = np.argmax(topics_predictions[0])

In [15]:
ctm.get_topic_lists(5)[topic_number]

['river', 'island', 'bridge', 'peak', 'line']

In [16]:
!wget https://raw.githubusercontent.com/vinid/data/master/italian_documents.txt

--2022-03-09 10:35:53--  https://raw.githubusercontent.com/vinid/data/master/italian_documents.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7122 (7.0K) [text/plain]
Saving to: ‘italian_documents.txt’

italian_documents.t 100%[===================>]   6.96K  --.-KB/s    in 0s      

2022-03-09 10:35:53 (58.4 MB/s) - ‘italian_documents.txt’ saved [7122/7122]



In [17]:
italian_documents = [line.strip() for line in open("italian_documents.txt", 'r').readlines()]
italian_documents

['Fu l\'ispiratore e uno dei fondatori della rivista "Dau al Set" e dell\'omonimo gruppo artistico ed intellettuale catalano (1948). È ritenuto il massimo esponente della poesia visiva non solo della letteratura catalana, ma il pioniere di questo genere in Spagna e uno dei grandi riferimenti internazionali. La totalità della sua opera creativa lo segnala come uno degli autori più prolifici della cultura occidentale contemporanea. Sebbene schierato nell\'avanguardia della poesia della prima metà del XX secolo (esplorò validamente l\'ipnagogia, il surrealismo e il dadaismo), si esercitò alla scrittura di centinaia di sonetti, odi saffiche e sestine liriche dalla totale perfezione formale alla sperimentazione più estrema, nonché migliaia di poemi in forma libera. In vita pubblicò un\'ottantina di libri lasciandone parecchi d\'inediti. Il suo lavoro letterario comprendeva tutti i generi: poesia, prosa, teatro (la sua cosiddetta "poesia scenica", oltre 350 opere), cinema, musica, cabaret, p

In [18]:
testing_dataset = tp.create_test_set(italian_documents)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
italian_topics_predictions = ctm.get_thetas(testing_dataset, n_samples=5)

/usr/local/lib/python3.7/dist-packages/contextualized_topic_models/models/ctm.py:257: DeprecationWarning: Call to `get_thetas` is deprecated and will be removed in version 2, use `get_doc_topic_distribution` instead
  DeprecationWarning)
Sampling: [5/5]: : 5it [00:00,  5.46it/s]


In [20]:
italian_documents[0]

'Fu l\'ispiratore e uno dei fondatori della rivista "Dau al Set" e dell\'omonimo gruppo artistico ed intellettuale catalano (1948). È ritenuto il massimo esponente della poesia visiva non solo della letteratura catalana, ma il pioniere di questo genere in Spagna e uno dei grandi riferimenti internazionali. La totalità della sua opera creativa lo segnala come uno degli autori più prolifici della cultura occidentale contemporanea. Sebbene schierato nell\'avanguardia della poesia della prima metà del XX secolo (esplorò validamente l\'ipnagogia, il surrealismo e il dadaismo), si esercitò alla scrittura di centinaia di sonetti, odi saffiche e sestine liriche dalla totale perfezione formale alla sperimentazione più estrema, nonché migliaia di poemi in forma libera. In vita pubblicò un\'ottantina di libri lasciandone parecchi d\'inediti. Il suo lavoro letterario comprendeva tutti i generi: poesia, prosa, teatro (la sua cosiddetta "poesia scenica", oltre 350 opere), cinema, musica, cabaret, pe

In [21]:
topic_number = np.argmax(italian_topics_predictions[0]) # get the topic id of the first document
ctm.get_topic_lists(10)[topic_number] 

['french',
 'published',
 'writer',
 'work',
 'war',
 'book',
 'children',
 'author',
 'books',
 'art']